In [6]:
%matplotlib inline
import pandas as pd

In [2]:
%%time
col_names=["frame", "x", "y", "obj","size","seq","tbd1","tbd2","tbd3","filename","time","path_time","delta_time","tbd4"]
use_cols=["x", "y", "obj","size","seq","filename","time","delta_time"]
df=pd.read_csv('data/fixed.csv',  names = col_names,usecols=use_cols, parse_dates=['time'],infer_datetime_format=True)


Wall time: 24.9 s


In [3]:
df['time'] = df['time'] + pd.to_timedelta(df['delta_time'])
df = df.drop(['delta_time'], axis=1)

In [4]:
gl_int = df.select_dtypes(include=['int64'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
df[converted_int.columns] = converted_int

In [7]:
gl_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]


compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)
df[converted_obj.columns] = converted_obj


In [8]:
df.head(10)

,x,y,obj,size,seq,filename,time
0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,4,291,1,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,629,7,2,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,253,4,3,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,185,4,4,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
5,5,102,5,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
6,6,102,5,362,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
7,6,102,5,362,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
8,7,102,5,362,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
9,9,102,5,362,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17


In [9]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 7 columns):
x           uint16
y           uint16
obj         uint16
size        uint32
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4), uint32(1)
memory usage: 62.9 MB


In [10]:
df.to_pickle("data/pickle.pkl.xz")